# 3.0 10X Genomics PBMC 2,700 Dataset

In [1]:
from clustergrammer2 import net

df = {}
import clustergrammer_groupby as cby
import gene_exp_10x

clustergrammer2 backend version 0.2.9


In [2]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline 

### Load Data

In [3]:
df = gene_exp_10x.load_gene_exp_to_df('../data/pbmc3k_filtered_gene_bc_matrices/hg19/')
df.shape

(32738, 2700)

### Remove Ribosomal and Mitochondrial Genes

In [4]:
all_genes = df.index.tolist()
print(len(all_genes))
keep_genes = [x for x in all_genes if 'RPL' not in x]
keep_genes = [x for x in keep_genes if 'RPS' not in x]
print(len(keep_genes))

df = df.loc[keep_genes]
df.shape

# Removing Mitochondrial Genes
list_mito_genes = ['MTRNR2L11', 'MTRF1', 'MTRNR2L12', 'MTRNR2L13', 'MTRF1L', 'MTRNR2L6', 'MTRNR2L7',
                'MTRNR2L10', 'MTRNR2L8', 'MTRNR2L5', 'MTRNR2L1', 'MTRNR2L3', 'MTRNR2L4']

all_genes = df.index.tolist()
mito_genes = [x for x in all_genes if 'MT-' == x[:3] or 
             x.split('_')[0] in list_mito_genes]
print(mito_genes)

keep_genes = [x for x in all_genes if x not in mito_genes]
df = df.loc[keep_genes]

32738
32546
['MTRNR2L11', 'MTRNR2L12', 'MTRNR2L13', 'MTRF1L', 'MTRNR2L6', 'MTRNR2L10', 'MTRNR2L7', 'MTRNR2L5', 'MTRNR2L8', 'MTRF1', 'MTRNR2L4', 'MTRNR2L1', 'MTRNR2L3', 'MT-ND1', 'MT-ND2', 'MT-CO1', 'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5', 'MT-ND6', 'MT-CYB']


In [5]:
def calc_mean_var_disp(df_inst):
    mean_arr = []
    var_arr = []
    mean_names = []
    for inst_gene in df_inst.index.tolist():
        mean_arr.append( df_inst.loc[inst_gene].mean() )
        var_arr.append(df_inst.loc[inst_gene].var())
        mean_names.append(inst_gene)

    ser_mean = pd.Series(data=mean_arr, index=mean_names)
    ser_var = pd.Series(data=var_arr, index=mean_names)    
    return ser_mean, ser_var

In [6]:
def cell_umi_count(df):
    sum_arr = []
    sum_names = []
    for inst_cell in df:
        sum_arr.append( df[inst_cell].sum() )
        sum_names.append(inst_cell)
    
    ser_sum = pd.Series(data=sum_arr, index=sum_names)
    return ser_sum

In [7]:
ser_sum = cell_umi_count(df)
df = df.div(ser_sum)
df.shape

(32520, 2700)

In [8]:
df.sum().head()

AAACATACAACCAC    1.0
AAACATTGAGCTAC    1.0
AAACATTGATCAGC    1.0
AAACCGTGCTTCCG    1.0
AAACCGTGTATGCG    1.0
dtype: float64

### Keep top 5000 expressing genes, ArcSinh Transform GEX Data

In [9]:
ser_mean, ser_var = calc_mean_var_disp(df)

keep_genes = ser_mean.sort_values(ascending=False)[:5000].index.tolist()

df = df.loc[keep_genes]

df = np.arcsinh(df/5)
df.shape

# save a copy of 5000 gene dataset for use with signatures
df_ini = deepcopy(df)
net.load_df(df_ini)
net.normalize(axis='row', norm_type='zscore')
df_ini = net.export_df()

print(df.shape)

(5000, 2700)


### Keep top 500 Genes by Variance

In [10]:
num_genes = 500
num_cells = 2700

# keep top genes based on variance
ser_mean, ser_var = calc_mean_var_disp(df)
keep_genes = ser_var.sort_values(ascending=False).index.tolist()[:num_genes]
df = df.loc[keep_genes]

# # randomly sample cells
# df = df.sample(axis=1, n=num_cells, random_state=99)

### Z-score GEX Across Cells

In [11]:
net.load_df(df)
net.normalize(axis='row', norm_type='zscore')
df = net.export_df()
df.shape

(500, 2700)

# Visualize Data in Clustergrammer2

In [12]:
net.load_df(df)
net.normalize(axis='row', norm_type='zscore')
net.clip(lower=-5, upper=5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "MALAT1", "ini": 500, "clust": 374, "rank": 484, "rankvar": 484…

### Load CIBERSORT gene sigantures

In [13]:
net.load_file('../data/cell_type_signatures/nm3337_narrow_cell_type_sigs.txt')
net.normalize(axis='row', norm_type='zscore')
df_sig = net.export_df()
print(df_sig.shape)

rows = df_sig.index.tolist()
new_rows = [x.split('_')[0] for x in rows]
df_sig.index = new_rows

(523, 22)


In [14]:
net.load_df(df_sig)
net.clip(lower=-5, upper=5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ABCB4", "ini": 523, "clust": 318, "rank": 341, "rankvar": 8, "…

# Predict Cell Types using NM3337 Signatures

In [15]:
df_pred_cat, df_sig_sim, y_info = cby.predict_cats_from_sigs(df_ini, df_sig, 
                                                                   predict_level='Cell Type', unknown_thresh=0.05)

In [16]:
df.columns = df_pred_cat.columns.tolist()

In [17]:
df_pred_cat.shape

(185, 2700)

In [18]:
net.load_df(df_pred_cat)
net.clip(lower=-5, upper=5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "PRF1", "ini": 185, "clust": 97, "rank": 106, "rankvar": 143, "…

### Cell Clustering Based on Top Genes by Variance with CIBERSORT Signature Categories

In [19]:
df.shape

(500, 2700)

In [20]:
net.load_df(df)
net.clip(lower=-5, upper=5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "MALAT1", "ini": 500, "clust": 374, "rank": 484, "rankvar": 484…